In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
import os
import shutil
import cv2
import numpy as np

# Укажите новые пути
IMAGES_TRAIN_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_img\train"
IMAGES_TEST_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_img\test"
MASKS_TRAIN_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_msk\train"
MASKS_TEST_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_msk\test"
OUTPUT_DIR = r'C:\Users\user\Desktop\nornik\split_orig_aug_data_yolo'

# Создание структуры папок
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

def convert_mask_to_yolo(mask_path, output_subdir):
    """Конвертирует маску в формат YOLO."""
    image = cv2.imread(mask_path)
    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return

    height, width = image.shape[:2]
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))
    new_image = np.ones_like(image) * 255
    new_image[black_mask > 0] = [0, 0, 0]
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, output_subdir, output_file_name)

    with open(output_file_path, 'w') as f:
        for contour in contours:
            contour_points = contour.reshape(-1, 2)
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

def process_data(images_dir, masks_dir, output_images_subdir, output_labels_subdir):
    """Копирует изображения и конвертирует маски."""
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
    for img in image_files:
        # Копирование изображения
        shutil.copy(os.path.join(images_dir, img), os.path.join(OUTPUT_DIR, output_images_subdir, img))
        # Конвертация маски
        mask_name = img.replace('.jpg', '.png')
        mask_path = os.path.join(masks_dir, mask_name)
        if os.path.exists(mask_path):
            convert_mask_to_yolo(mask_path, output_labels_subdir)
        else:
            print(f"Маска отсутствует для изображения: {img}")

# Обработка обучающего и тестового набора
process_data(IMAGES_TRAIN_DIR, MASKS_TRAIN_DIR, 'images/train', 'labels/train')
process_data(IMAGES_TEST_DIR, MASKS_TEST_DIR, 'images/val', 'labels/val')

# Создание файла data.yaml
data_yaml_content = f"""
train: {os.path.join(OUTPUT_DIR, 'images/train')}
val: {os.path.join(OUTPUT_DIR, 'images/val')}

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

yaml_path = os.path.join(OUTPUT_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"Датасет успешно обработан и сохранен в: {OUTPUT_DIR}")

Датасет успешно обработан и сохранен в: C:\Users\user\Desktop\nornik\split_orig_data_yolo


In [3]:
model = YOLO("yolo11n-seg.pt")

  0%|          | 0.00/5.90M [00:00<?, ?B/s]

In [ ]:
import torch


torch.set_num_threads(1)

train_results = model.train(
    data=r"C:\Users\user\Desktop\nornik\split_orig_data_yolo\data.yaml",  # path to dataset YAML
    epochs=1000,  # number of training epochs
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    patience=500,
)

New https://pypi.org/project/ultralytics/8.3.44 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.38  Python-3.10.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24564MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=C:\Users\user\Desktop\nornik\split_orig_data_yolo\data.yaml, epochs=500, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize

  0%|          | 0.00/5.35M [00:00<?, ?B/s]

AMP: checks passed 


train: Scanning C:\Users\user\Desktop\nornik\split_orig_data_yolo\labels\train... 200 images, 131 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<00:00, 860.01it/s]

train: New cache created: C:\Users\user\Desktop\nornik\split_orig_data_yolo\labels\train.cache


...

    318/500      2.99G     0.6022      1.164     0.5126     0.9562         19        640: 100%|██████████| 13/13 [00:02<00:00,  6.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.40it/s]

                   all         50         42      0.651      0.738      0.746      0.433      0.672      0.762      0.761      0.396
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 218, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



318 epochs completed in 0.222 hours.
Optimizer stripped from runs\segment\train\weights\last.pt, 6.0MB
Optimizer stripped from runs\segment\train\weights\best.pt, 6.0MB

Validating runs\segment\train\weights\best.pt...
Ultralytics 8.3.38  Python-3.10.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24564MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.60it/s]


                   all         50         42      0.746      0.698      0.787      0.473      0.861      0.571      0.732      0.414
Speed: 0.4ms preprocess, 1.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\segment\train


In [ ]:
results = model.val(data=r"C:\Users\user\Desktop\norn\2 задача\2 задача\train_dataset\baseline\data.yaml",)
print(results)

Ultralytics 8.3.44  Python-3.10.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24564MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning C:\Users\user\Desktop\norn\2 задача\2 задача\train_dataset\baseline\train_data\labels\val.cache... 34 images, 24 backgrounds, 0 corrupt: 100%|██████████| 34/34 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.90it/s]


                   all         34         22      0.841      0.481      0.616      0.463      0.516      0.636      0.552      0.371
Speed: 1.1ms preprocess, 14.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\segment\train72
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021CF7586440>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.0220